# Maestría en "Analítica de datos"
---
Nombre: Jader Antonio Gomez Orrego
Código: 1018422976
Fecha: 2025 agosto 28
---
Descripción: este libro trabaja ejercicios de "analítica/ETL/estadística descriptiva"

# 1.Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.carga/obtención de **información**

In [3]:
file_path_csv="/content/drive/MyDrive/BigData/Dataset's/Población_BDUA_Regimen/Población_Base_de_Datos_Única_de_Afiliados_BDUA_del_régimen_contributivo_20250922.csv"
try:
  df=pd.read_csv(file_path_csv)
except FileNotFoundError:
  print(f"El archivo {file_path_csv} no fue encontrado.")
except Exception as e:
  print(f"Ocurrió un error al leer el archivo: {e}")

In [4]:
pd.set_option("display.max_columns", None)
display(df.head())

,Género,Grupo etario,Código de la entidad,Nombre de la entidad,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros
0,Femenino,1 a 5,EPS037,NUEVA EPS S.A.,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,ANTIOQUIA,BETULIA,N,VÍCTIMAS DEL CONFLICTO ARMADO INTERNO,2
1,Masculino,15 a 19,EPS041,NUEVA EPS S.A. -CM,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,HUILA,PITALITO,1,POBLACIÓN CON SISBEN,2
2,Femenino,15 a 19,EPS041,NUEVA EPS S.A. -CM,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,BOYACA,SOGAMOSO,NaN,NaN,13
3,Femenino,50 a 55,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CORDOBA,LA APARTADA,NaN,NaN,3
4,Femenino,60 a 65,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CUNDINAMARCA,ARBELAEZ,1,POBLACIÓN CON SISBEN,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641321 entries, 0 to 641320
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Género                      641321 non-null  object
 1   Grupo etario                641321 non-null  object
 2   Código de la entidad        641321 non-null  object
 3   Nombre de la entidad        641321 non-null  object
 4   Régimen                     641321 non-null  object
 5   Tipo de afiliado            641321 non-null  object
 6   Estado del afiliado         641321 non-null  object
 7   Condición del beneficiario  641321 non-null  object
 8   Zona de Afiliación          641321 non-null  object
 9   Departamento                641321 non-null  object
 10  Municipio                   641321 non-null  object
 11  tps_nvl_ssb_id              360387 non-null  object
 12  Nivel del Sisbén            360313 non-null  object
 13  Cantidad de registros       6

In [6]:
df.describe()

,Género,Grupo etario,Código de la entidad,Nombre de la entidad,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros
count,641321,641321,641321,641321,641321,641321,641321,641321,641321,641321,641321,360387,360313,641321
unique,2,12,31,31,1,3,2,3,6,33,1035,4,19,3023
top,Masculino,19 a 45,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,ANTIOQUIA,SANTIAGO DE CALI,1,POBLACIÓN CON SISBEN,1
freq,324586,151690,272932,272932,641321,329677,594972,612965,308602,80836,6093,139814,238525,278471


# 3.ETL

In [11]:
# Convert 'Cantidad de registros' to numeric, coercing errors, then to integer
df['Cantidad de registros'] = pd.to_numeric(df['Cantidad de registros'], errors='coerce')
df['Cantidad de registros'] = df['Cantidad de registros'].fillna(0).astype(int)
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641321 entries, 0 to 641320
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Género                      641321 non-null  object
 1   Grupo etario                641321 non-null  object
 2   Código de la entidad        641321 non-null  object
 3   Nombre de la entidad        641321 non-null  object
 4   Régimen                     641321 non-null  object
 5   Tipo de afiliado            641321 non-null  object
 6   Estado del afiliado         641321 non-null  object
 7   Condición del beneficiario  641321 non-null  object
 8   Zona de Afiliación          641321 non-null  object
 9   Departamento                641321 non-null  object
 10  Municipio                   641321 non-null  object
 11  tps_nvl_ssb_id              360387 non-null  object
 12  Nivel del Sisbén            360313 non-null  object
 13  Cantidad de registros       6

None

#4.estadísticas descriptivas

In [12]:
df['Género'].value_counts()

,count
Género,
Masculino,324586
Femenino,316735


In [14]:
df_eps = df.groupby('Nombre de la entidad')['Cantidad de registros'].sum()
#calcular porcentaje
total_registros = df['Cantidad de registros'].sum()
entidad_porcentaje = (df_eps / total_registros) * 100
resultado_df = pd.DataFrame({'Cantidad de Registros': df_eps, 'Porcentaje': entidad_porcentaje})
resultado_df = resultado_df.sort_values(by='Porcentaje', ascending=False)
display(resultado_df.head())
#display(df_eps.head())

,Cantidad de Registros,Porcentaje
Nombre de la entidad,,
NUEVA EPS S.A.,3213637,38.147962
ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.,940898,11.169072
SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A.,888917,10.552023
EPS FAMISANAR S.A.S.,625881,7.429615
EPS SURAMERICANA S.A.,623137,7.397042


# 5.creación de BD SQLITE y tablas

In [17]:
db_path="//content/drive/MyDrive/BigData/Dataset's/SQLITE/dbua.db"
db_dir =os.path.dirname(db_path)
if os.path.exists(db_dir):
  print(f"El directorio {db_dir} existe.")
else:
  os.makedirs(db_dir)
  print(f"El directorio {db_dir} fue creado.")

El directorio //content/drive/MyDrive/BigData/Dataset's/SQLITE existe.


In [18]:
try:
  conn=sqlite3.connect(db_path)
  cursor=conn.cursor()          #objeto que va a interactiar con la bd (dml/ddl)
  print("conectado")
except sqlite3.Error as e:
  print(f"Error al conectar a la base de datos: {e}")


conectado


## 5.1 creando tablas

In [19]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Departamento (
    IdDepartamento INTEGER PRIMARY KEY AUTOINCREMENT,
    NombreDepartamento TEXT UNIQUE NOT NULL
)
''')

# SIGUIENTE CLASE CONTINUAMOS.....

In [21]:
!pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 85.7 MB/s eta 0:00:00


In [23]:
df_tutelas_eps = df.copy()
df_tutelas_eps = df_tutelas_eps.rename(columns={'Nombre de la entidad': 'Nombre_EPS'})

# Add a placeholder for the 'demandado_limpio' column.
# You will need to populate this column with the relevant data for fuzzy matching.
df_tutelas_eps['demandado_limpio'] = ""

display(df_tutelas_eps.head())

,Género,Grupo etario,Código de la entidad,Nombre_EPS,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros,demandado_limpio
0,Femenino,1 a 5,EPS037,NUEVA EPS S.A.,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,ANTIOQUIA,BETULIA,N,VÍCTIMAS DEL CONFLICTO ARMADO INTERNO,2,
1,Masculino,15 a 19,EPS041,NUEVA EPS S.A. -CM,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,HUILA,PITALITO,1,POBLACIÓN CON SISBEN,2,
2,Femenino,15 a 19,EPS041,NUEVA EPS S.A. -CM,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,BOYACA,SOGAMOSO,NaN,NaN,13,
3,Femenino,50 a 55,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CORDOBA,LA APARTADA,NaN,NaN,3,
4,Femenino,60 a 65,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CUNDINAMARCA,ARBELAEZ,1,POBLACIÓN CON SISBEN,1,


In [26]:
from fuzzywuzzy import process

# Crear un diccionario para almacenar los nombres de EPS mapeados
eps_mapping = {}

# Obtener nombres de demandados limpios únicos de df_tutelas_eps
unique_demandados_limpios = df_tutelas_eps['demandado_limpio'].unique()

# Obtener la lista de nombres estándar de EPS de df_eps. The index contains the EPS names.
standard_eps_names = df_eps.index.tolist()

# Iterar a través de nombres de demandados limpios únicos y encontrar la mejor coincidencia en nombres estándar de EPS
for demandado_limpio in unique_demandados_limpios:
    if pd.notna(demandado_limpio):
        # Encontrar la mejor coincidencia usando coincidencia aproximada (el umbral de puntuación se puede ajustar)
        best_match, score = process.extractOne(demandado_limpio, standard_eps_names)
        # Si la puntuación está por encima de un cierto umbral, considerarlo una coincidencia
        if score > 80:  # Ajustar umbral según sea necesario
            eps_mapping[demandado_limpio] = best_match
        else:
            eps_mapping[demandado_limpio] = None # No se encontró una buena coincidencia

# Crear una nueva columna en df_tutelas_eps con los nombres de EPS mapeados
df_tutelas_eps['mapped_eps'] = df_tutelas_eps['demandado_limpio'].map(eps_mapping)

# Inicializar total_tutelas en df_eps a cero
# df_eps['total_tutelas'] = 0 # df_eps is a Series, not a DataFrame, cannot add a column this way.

# Create a new DataFrame to store the total tutelas per EPS
total_tutelas_per_eps = df_tutelas_eps.groupby('mapped_eps').size().reset_index(name='total_tutelas')

# Merge the total tutelas with the df_eps Series (which is now the index of resultado_df)
# The merge needs to be done with resultado_df, which is the DataFrame containing df_eps data.
# Let's assume resultado_df is available and use it for the merge. If not, we might need to recreate it.
# Assuming resultado_df is available:
if 'resultado_df' in locals():
    resultado_df = resultado_df.merge(total_tutelas_per_eps, left_index=True, right_on='mapped_eps', how='left')
    resultado_df['total_tutelas'] = resultado_df['total_tutelas'].fillna(0).astype(int)
    resultado_df = resultado_df.drop(columns=['mapped_eps']) # Drop the redundant column after merge
else:
    # If resultado_df is not available, create a new DataFrame from df_eps and total_tutelas_per_eps
    resultado_df = pd.DataFrame({'Cantidad de Registros': df_eps})
    resultado_df = resultado_df.merge(total_tutelas_per_eps, left_index=True, right_on='mapped_eps', how='left')
    resultado_df['total_tutelas'] = resultado_df['total_tutelas'].fillna(0).astype(int)
    resultado_df = resultado_df.drop(columns=['mapped_eps']) # Drop the redundant column after merge
    # Recalculate percentage if resultado_df was not available and needed recreation
    total_registros = resultado_df['Cantidad de Registros'].sum()
    resultado_df['Porcentaje'] = (resultado_df['Cantidad de Registros'] / total_registros) * 100


# Mostrar el resultado_df actualizado con el total de tutelas y el porcentaje
display(resultado_df.head())

# Mostrar demandados que no pudieron ser mapeados
unmapped_demandados = df_tutelas_eps[df_tutelas_eps['mapped_eps'].isna()]['Nombre_EPS'].unique() # Use 'Nombre_EPS' since 'demandado' is not in the current df
print(f"\nDemandados que no pudieron ser mapeados: {len(unmapped_demandados)}")
for demandado in unmapped_demandados:
    print(demandado)

,Cantidad de Registros,Porcentaje,total_tutelas
Nombre de la entidad,,,
NUEVA EPS S.A.,3213637,38.147962,0
ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.,940898,11.169072,0
SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A.,888917,10.552023,0
EPS FAMISANAR S.A.S.,625881,7.429615,0
EPS SURAMERICANA S.A.,623137,7.397042,0



Demandados que no pudieron ser mapeados: 31
NUEVA EPS S.A.
NUEVA EPS S.A. -CM
EMSSANAR S.A.S. -CM
CAPRESOCA E.P.S. -CM
EPS SURAMERICANA S.A.
SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A.
ENTIDAD PROMOTORA DE SALUD SERVICIO OCCIDENTAL DE SALUD S.A. S.O.S.
ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS
CAJACOPI EPS S.A.S -CM
ASMET SALUD EPS S.A.S. -CM
EPS FAMISANAR S.A.S.
ENTIDAD PROMOTORA DE SALUD MALLAMAS EPSI -CM
COOSALUD EPS S.A. -CM
CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
PIJAOS SALUD EPSI -CM
ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.
EPS FAMILIAR DE COLOMBIA S.A.S. -CM
ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS -CM
CAJA DE COMPENSACION FAMILIAR DEL VALLE DEL CAUCA COMFENALCO VALLE DE LA GENTE
COOSALUD EPS S.A.
CAPITAL SALUD ENTIDAD PROMOTORA DE SALUD DEL RÉGIMEN SUBSIDIADO SAS CAPITAL SALUD EPS-S S.A.S. -CM
FUNDACIÓN SALUD MIA
ALIANZA MEDE